<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2022/images/logo.png?raw=true" alt="2022年度ゲノム情報解析入門" height="100px" align="middle">

<div align="right"><a href="https://github.com/CropEvol/lecture#section2">実習表ページに戻る</a></div>

QTL-seq入門 (3): SNP-indexの算出、グラフ描画
---

　前回・前々回に続き、次世代シーケンサーを使った遺伝子マッピング手法のひとつ、QTL-seqを勉強・体験していきます。

QTL-seq入門シリーズ: 
- [QTL-seq入門 (1): 参照配列（リファレンス配列）の準備](https://colab.research.google.com/github/CropEvol/lecture/blob/master/textbook_2022/QTLseq_1.ipynb)
- [QTL-seq入門 (2): リードアライメント、SNP検出](https://colab.research.google.com/github/CropEvol/lecture/blob/master/textbook_2022/QTLseq_2.ipynb)
- QTL-seq入門 (3): SNP-indexの算出、グラフ描画 ←今回


<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2022/images/qtlseq_workflow.png?raw=true" alt="QTL-seq workflow" height="180px" align="middle">

In [ ]:
#########################
##  実習の前に実行してください。 ##
#########################
## Colab上にデータファイルのダウンロードする
!wget -q -O rice_qtlseq_dataset.csv https://github.com/CropEvol/lecture/raw/master/textbook_2022/datasets/rice_qtlseq_dataset.csv
!ls | grep "rice"

# 今回の勉強内容

0. 前回の復習
1. アライメント結果からSNPデータセットを作る
1. SNPデータセットの扱い方
  1. データセットを読み込む
  1. 列データを取り出す
  1. 列データで計算をする
  1. 任意の染色体のデータ行を取り出す
1. グラフ描画
  1. グラフ描画の基礎
  1. 散布図を描く
  1. Slinding window解析
  1. 原因遺伝子の位置を探す


# 0. 前回の復習

　前回、イネのQTL-seqのリードアライメント結果をみました。

イネのQTL-seqの概要: 
- いもち病への抵抗性品種と罹病性品種の交配分離集団
- バルクシーケンシング:
  - 抵抗性個体のバルク
  - 罹病性個体のバルク
- 参照配列: 罹病性品種のゲノム配列
- リードアライメント結果: [IGV-Web 版](https://igv.org/app/?sessionURL=blob:zZJLb8IwDID_i89dHwy2tTck2GmgaQ9pEpoqt3VpRNq0TgqbEP99HrAT7MakSckljv3ZX7KFNbFVpoEEBn507Y_AA1uZzTPWraY51mQhKVFb8oCpJKYmJ0i2UKJ1.Pr0IImVc61NgoBx4y.Vq_qst8S5aRw1zs9NHXQdtTd3oyKw.7JpgQ4tuaBG1QSdk.pdwCqnNK_4Jo3SaDTL_BKlF9UU9PHHGFkKdh5ok_cyLXxHk.gqCr1wEEsPbBQki3cPHGO.khuLLbjPVjSAVOz3RjwwXBBDchWH4W0Ux4PR8HYYxnG087bQs77YAM9p1uuVf7jhZ1gLu5F3EsC50MUFnkBkKwHJ9GpNxXzfiuNePkxpuEYn5EMrR2eo1bKpBSlHqDVpumfqXiom.Xi6gCT0B1JO2Vbj58wU3znTt8fxfDKdSMpa0WZsH1Gx_eEc1Q8vrfrpd9XnQhdXfQL5R6rfd18-) <small>※ Firefoxではエラーが出て見れません</small>
  

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2022/images/qtlseq_takagi_etal_2013_L06.png?raw=true" alt="Rice QTL-seq example (Takagi et al., 2013)" height="250px" align="middle">


　今回、リードアライメント上で見つけることができるSNP情報のデータセットをPythonで扱い、抵抗性-罹病性の違いを作っている原因遺伝子のマッピングをしていきます。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2022/images/qtlseq_data_analysis.png?raw=true" alt="qtlseq_data_analysis" height="180px" align="middle">


# 1. アライメント結果からSNPデータセットを作る

　ゲノム上のSNP情報を、Pythonなどで扱うには、Pythonで読み込みやすい形式に変換する必要があります。どのようにするか？ 詳細は省きますが、各種ソフトウェア（[bcftools](https://samtools.github.io/bcftools/bcftools.html)など）で変換できます。

　ここでは、ソフトウェアを使って、「リードアライメントからSNP情報のみを抜き出したデータセットを作れる」ことを理解してもらえればOKです。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2022/images/alignment_to_dataset.png?raw=true" alt="alignment to SNP dataset" height="160px" align="middle">

# 2. SNPデータセットの扱い方

　ここからは、用意したSNPデータセットをどのようにPythonで扱うかを勉強していきましょう。


- イネQTL-seq SNPデータセット [rice_qtlseq_dataset.csv](https://raw.githubusercontent.com/CropEvol/lecture/master/textbook_2022/datasets/rice_qtlseq_dataset.csv)

## 2-1. データセットを読み込む

　SNPデータセットは、1行あたり8個のデータがカンマ記号(`,`)で区切られた、テーブル状の構造をしています。カンマ区切りのテーブル状データセットは、pandasと呼ばれるライブラリ（Pythonの追加機能のようなツール）の`read_csv`で読み込むことができ、その後、データ処理をおこなえるようになります。

```python
# データセットを読み込む
import pandas as pd
データフレーム変数 = pd.read_csv("ファイル名")
```

In [ ]:
import pandas as pd
df = pd.read_csv("rice_qtlseq_dataset.csv")
df

　読み込んだデータセットの見方は下図のとおりです。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2022/images/pandas_dataframe.png?raw=true" alt="pandas_dataframe" height="320px" align="middle">

## 2-2. 列データを取り出す

　まず最初に、1列分のデータの取り出し方を学びましょう。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2022/images/extract_one_column.png?raw=true" alt="extract_one_column" height="120px" align="middle">

```python
# 列データを取り出す
データフレーム変数["列名"]
　　　　または
データフレーム変数.列名
```

In [ ]:
df["Bulk1_REF"]

### 練習2-2
　`Bulk1_ALT`の列を取り出してください。

In [ ]:
## 練習2-2: 「Bulk1_ALT」列を取り出す


#### 解答例

In [ ]:
df["Bulk1_ALT"]
df.Bulk1_ALT

## 2-3. 列データで計算する

　次に、列データを使って、SNP-indexの算出をおこないましょう。列データと列データの要素同士の計算は、次のようなコードで簡単におこなえます。

```python
# 列データ同士を計算する（足し算の場合）
列Aのデータ + 列Bのデータ
# 新しいデータとして追加する
データフレーム変数["新しい列名"] = 列Aのデータ + 列Bのデータ
```

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2022/images/pandas_calc_columns.png?raw=true" alt="pandas_calc_columns" height="200px" align="middle">

　ここでは、Bulk1のSNP-indexを計算してみましょう。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2022/images/calculate_snp_index_of_bulk1.png?raw=true" alt="calculate_snp_index" height="60px" align="middle">

In [ ]:
df["Bulk1_SNPindex"] = df["Bulk1_ALT"]/(df["Bulk1_REF"] + df["Bulk1_ALT"])
df

### 練習2-3
　Bulk2のSNP-indexも算出して、データフレーム`df`の新しい列`Bulk2_SNPindex`に追加してください。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2022/images/calculate_snp_index_of_bulk2.png?raw=true" alt="calculate_snp_index" height="60px" align="middle">

In [ ]:
## 練習2-3: Bulk2のSNPindexを計算し、dfに「Bulk2_SNPindex」列を追加する
df["Bulk2_SNPindex"] = 
df

#### 解答例

In [ ]:
df["Bulk2_SNPindex"] = df["Bulk2_ALT"]/(df["Bulk2_REF"] + df["Bulk2_ALT"])
df

## 2-4. 任意の染色体のデータ行を取り出す

　「2-3. 列データで計算する」で、各バルクについて、ゲノム全域のSNP-indexを算出しました。

　このあと、任意の染色体（例えば、`chr01`）のSNP-indexのグラフを描くために、任意の染色体の情報のみを取り出す方法を学びましょう。

```python
# 条件に合ったデータ行を取り出す
データフレーム変数[条件]
# 例: データフレーム「df」から、列「CHROM」が"chr01"の行を取り出す
df[df["CHROM"]=="chr01"]
```

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2022/images/pandas_filtering_data.png?raw=true" alt="pandas_filtering_data" height="200px" align="middle">

In [ ]:
sub = df[df["CHROM"]=="chr01"]
sub

### 練習2-4
　データフレーム「df」から、「CHROM」列がchr12のデータを取り出してください。

In [ ]:
## 練習2-4: dfから「CHROM」列がchr12のデータのみを取り出す
sub12 = 
sub12

#### 解答例

In [ ]:
sub12 = df[df["CHROM"]=="chr12"]
sub12

# 3. グラフ描画

　次に、グラフ描画の方法を学び、SNP-indexをグラフにしていきましょう。

## 3-1. グラフ描画の基礎

　Pythonでのグラフ描画の基礎は次のとおりです。
> グラフのパーツが描かれた透明な層（レイヤー）を複数枚かさねて、ひとつのグラフを描く

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2022/images/drawing_plot_basis.png?raw=true" alt="drawing_plot_basis" height="180px" align="middle">


## 3-2. 散布図を描く

　`pandas`とは別のライブラリ`matplotlib`（グラフ描画用ライブラリ）を使うと、様々なグラフを描けます。今回は、SNP-indexの散布図を描いてみましょう。

```python
x = x座標データ
y = y座標データ

# 散布図
import matplotlib.pyplot as plt
plt.scatter(x, y, オプション（任意）)
```

　ここでの例として、データフレーム`sub`の情報（ひとつの染色体の情報）を使って、Bulk1のSNP-indexの散布図を描きます。

- x軸座標データ: `POS`列
- y軸座標データ: `Bulk1_SNPindex`列

In [ ]:
## x軸、y軸データ
x  = sub["POS"]
y1 = sub["Bulk1_SNPindex"]

import matplotlib.pyplot as plt
plt.figure(figsize=[12, 4])  # グラフフィールド（サイズ指定）
plt.scatter(x, y1)          # 散布図
plt.xlabel("Position (x 10 Mbp)") # x軸ラベル
plt.ylabel("SNP index of Bulk1")  # y軸ラベル
plt.show()                 # 表示

### 練習3-2

　10行目にコードを追記して、Bulk2のSNP-indexの散布図を描いてください。

In [ ]:
## 練習3-2: Bulk2のSNP-indexの散布図を描く
## x軸、y軸データ
x  = sub["POS"]
y2 = sub["Bulk2_SNPindex"]

import matplotlib.pyplot as plt
plt.figure(figsize=[12, 4])  # グラフフィールド（サイズ指定）
## ------- 編集箇所1 ------------
# 散布図
plt.scatter()
## ---------------------------------
plt.xlabel("Position (x 10 Mbp)") # x軸ラベル
plt.ylabel("SNP index of Bulk2")  # y軸ラベル
plt.show()                # 表示

#### 解答例

In [ ]:
## 練習3-2: Bulk2のSNP-indexの散布図を描く
## x軸、y軸データ
x  = sub["POS"]
y2 = sub["Bulk2_SNPindex"]

import matplotlib.pyplot as plt
plt.figure(figsize=[12, 4])  # グラフフィールド（サイズ指定）
## ------- 編集箇所1 ------------
# 散布図
plt.scatter(x, y2)
## ---------------------------------
plt.xlabel("Position (x 10 Mbp)") # x軸ラベル
plt.ylabel("SNP index in Bulk2")  # y軸ラベル
plt.show()                # 表示

## 3-3. Slinding window解析

　散布図だけでは、原因遺伝子がありそうな場所を判断するのは難しそうです。

　バルクシーケンシングで得られるデータ（SNPアリルの個数、また、そこから求められるSNP-index）は、ある程度ランダム性を持っています。ALT塩基を50%で得られることが期待できる場所（SNP-indexが0.5になる場所）では、その期待値周辺のSNP-indexが得られますが、ときには大きく外れたSNP-indexが得られることもあります。

<small>※ コイントスを例にすると、「裏面が出る確率50%のコインを10枚投げて、裏面が出た枚数を調べる」という何度も繰り返すと、裏が5枚出る試行が多く観察されますが、10枚とも裏が出たり、裏が0枚の試行も生じることがあります。SNP-indexも同じように、多くのSNPを得ると、期待値付近のSNP-indexが多く得られますが、大きく外れたSNP-indexも得られることがあります。</small>


　期待SNP-indexは、周辺のSNP座の平均SNP-indexで近似できます。任意の区間を設定し、その区間の平均SNP-indexを調べるといった作業を、ゲノム全域にわたっておこなうと、平均SNP-indexが高い場所や低い場所を見つけることができます。

　ゲノム全域にわたってSNP-indexの平均値の推移（**移動平均**）を調べる解析法のことを**Slinding window解析**と言います。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2022/images/sliding_window_overview.png?raw=true" alt="sliding_window_overview" height="350px" align="middle">

　移動平均は、列データに対して`rolling`という関数を使うと、簡単に得られます。

```python
# 任意SNP数区間の移動平均を求める
列データ.rolling(データ数).mean()
```

　また、移動平均の折れ線グラフはplot関数を使うことで描けます。
```python
# 移動平均の折れ線グラフを追加する
import matplotlib.pyplot as plt
plt.scatter(x, y, オプション（任意）)   # x=x座標データ, y=y座標データ
plt.plot(win_x, win_y, オプション（任意）) # win_x=x座標データの移動平均, win_y=y座標データの移動平均
```

In [ ]:
## Bulk1_SNPindexの移動平均
win_x  = sub["POS"].rolling(20).mean()             #POS列（SNP 20個分）の移動平均
win_y1 = sub["Bulk1_SNPindex"].rolling(20).mean()  #Bulk1_SNPindex列（SNP 20個分）の移動平均

## 移動平均をグラフ化する
import matplotlib.pyplot as plt
plt.figure(figsize=[12, 3])               #　グラフフィールド（サイズ指定）
plt.scatter(x, y1)                       # 散布図
plt.plot(win_x, win_y1, color="orange") # 移動平均の線グラフ
plt.xlabel("Position (x 10 Mbp)")         # x軸ラベル
plt.ylabel("SNP index of Bulk1")         # y軸ラベル
plt.show()                               # 表示

In [ ]:
## Bulk2_SNPindexの移動平均
win_x  = sub["POS"].rolling(20).mean()             #POS列（SNP 20個分）の移動平均
win_y1 = sub["Bulk2_SNPindex"].rolling(20).mean()  #Bulk2_SNPindex列（SNP 20個分）の移動平均

## 移動平均をグラフ化する
import matplotlib.pyplot as plt
plt.figure(figsize=[12, 3])               #　グラフフィールド（サイズ指定）
plt.scatter(x, y1)                       # 散布図
plt.plot(win_x, win_y2, color="orange") # 移動平均の線グラフ
plt.xlabel("Position (x 10 Mbp)")         # x軸ラベル
plt.ylabel("SNP index of Bulk1")         # y軸ラベル
plt.show()                               # 表示

## 3-4. 原因遺伝子の位置を探す

　イネには第1〜12染色体があります。次の二つのコードセルのプログラムを実行すると、任意の（二つ目のコードセルの`chrom`で指定した）染色体のSNP-indexをグラフ描画できます。それぞれの染色体のSNP-indexグラフを調べて、原因遺伝子がありそうな場所を探してください。

なお、ここで解析しているイネQTL-seqの参照配列とバルク、SNP-indexの算出は以下のとおりであることを思い出してください。
- 参照配列: 罹病性イネ品種のゲノム配列
- 分離集団のバルク構成:
  - Bulk1: 抵抗性個体を集めたグループ
  - Bulk2: 罹病性個体を集めたグループ
- SNP-index: 抵抗性品種由来のSNPアリルの割合


In [ ]:
## グラフ描画用の関数
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
mpl.style.use('seaborn-darkgrid')
def draw_QTLseq(data, window_size=20, fs=20, lw=4, a=0.3):
    ## グラフ用データ
    x = data["pos"]/1000000  # x座標データ: POS (x Mbp)
    y1= data["bulk1"]  # サブグラフ1のy座標データ: Bulk1 SNP-index
    y2= data["bulk2"]  # サブグラフ2のy座標データ: Bulk2 SNP-index
    y3= data["delta"]  # サブグラフ3のy座標データ: delta SNP-index
    win_x = x.rolling(window_size).mean()   # POS移動平均
    win_y1= y1.rolling(window_size).mean() # Bulk1 SNP-index移動平均
    win_y2= y2.rolling(window_size).mean() # Bulk2 SNP-index移動平均
    win_y3= y3.rolling(window_size).mean() # delta SNP-index移動平均

    ## グラフ設定
    fig = plt.figure(figsize=[12, 8])
    ax1 = fig.add_subplot(3,1,1) # サブグラフ1
    ax2 = fig.add_subplot(3,1,2) # サブグラフ2
    ax3 = fig.add_subplot(3,1,3) # サブグラフ3
    ## サブグラフ1（Bulk1のグラフ）
    ax1.scatter(x, y1, c="#636EFA", alpha=a)
    ax1.plot(win_x, win_y1, c="#EF553B", linewidth=lw)
    ax1.set_ylabel("SNP-index\nof Bulk1", fontsize=fs)
    ax1.tick_params(labelsize=fs)
    ax1.set_ylim([-0.1, 1.1])
    ## サブグラフ2（Bulk2のグラフ）
    ax2.scatter(x, y2, c="#00CC96", alpha=a)
    ax2.plot(win_x, win_y2, c="#AB63FA", linewidth=lw)
    ax2.set_ylabel("SNP-index\nof Bulk2", fontsize=fs)
    ax2.tick_params(labelsize = fs)
    ax2.set_ylim([-0.1, 1.1])
    ## サブグラフ3（delta-indexのグラフ）
    ax3.scatter(x, y3, c="#FFA15A", alpha=a)
    ax3.plot(win_x, win_y3, c="#19D3F3", linewidth=lw)
    ax3.set_xlabel("Position (x Mbp)", fontsize=fs)
    ax3.set_ylabel("delta of\nSNP-index", fontsize=fs)
    ax3.tick_params(labelsize = fs)
    ax3.set_ylim([-1.1, 1.1])
    plt.tight_layout()
    plt.show()

In [ ]:
## 表示する染色体番号（1〜12の数字を入力可能）
chrom = 1

## ---------------------------
## データセット
import pandas as pd
df2  = pd.read_csv("rice_qtlseq_dataset.csv", sep=",", header=0)
sub2 = df[df["CHROM"]=="chr%02d" % chrom]
## SNP-index算出
position     = sub2["POS"]
bulk1_index = sub2["Bulk1_ALT"] / (sub2["Bulk1_REF"] + sub2["Bulk1_ALT"])
bulk2_index = sub2["Bulk2_ALT"] / (sub2["Bulk2_REF"] + sub2["Bulk2_ALT"])
delta_index = bulk1_index - bulk2_index
data = {"pos": position, "bulk1": bulk1_index, "bulk2": bulk2_index, "delta": delta_index}
## グラフ描画
draw_QTLseq(data)

### 原因遺伝子座

　第6染色体の3~5Mbのあたりに、バルク間で平均SNP-index（抵抗性アリルの割合）に大きな差がみられます。
- 抵抗性バルク（Bulk1）で平均SNP-indexが高い
- 罹病性バルク（Bulk2）で平均SNP-indexが低い
- delta SNP-indexが高い

　そのゲノム領域に抵抗性-罹病性を決定する遺伝子があると考えられます。

---
# まとめ

　このテキストでは、Pythonを使って、SNP-indexの算出とグラフ描画をおこないました。

- PythonでSNPデータセットを読み込む
- SNPアリルの個数からSNP-indexを算出する
- SNP-indexの散布図を描く
- SNP-indexの移動平均の折れ線グラフを描く

　また、これまでの三回のQTL-seq入門シリーズを通して、次世代シーケンス解析の基礎も勉強してきました。
1. 参照配列の準備 → **ゲノムアセンブル**
1. **リードアライメント**と**SNP検出**
1. SNP-index算出とグラフ描画 → Pythonを使った**SNPデータ解析**

　これまで学んできたことを、みなさんの今後の勉強にぜひ役立ててください。






<div align="right"><a href="https://github.com/CropEvol/lecture#section2">実習表ページに戻る</a></div>